<a href="https://colab.research.google.com/github/twidddj/tf-reformer/blob/master/reformer_(duplication_task).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/twidddj/tf-reformer

Cloning into 'tf-reformer'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 1), reused 22 (delta 1), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [3]:
import os
import sys

import numpy as np
import tensorflow as tf
from time import time

sys.path.append('./tf-reformer')
from dupltask.train import DuplTaskReformer, get_sample, log_dir_tmpl, log_dir_full_attn_tmpl
log_dir_tmpl = os.path.join('./tf-reformer/dupltask', log_dir_tmpl)

In [5]:
get_sample(64, 15) # vocab_size: 64, seq_len:32, seg_len: 15

array([ 0, 33, 24, 58, 15, 33, 17, 35, 46, 22,  1, 32, 28, 61, 25, 14,  0,
       33, 24, 58, 15, 33, 17, 35, 46, 22,  1, 32, 28, 61, 25, 14])

# config

In [6]:
N = 8
d_model = 128
d_ff = 256
num_heads = 4
num_blocks = 1
vocab_size = 64
seq_len = 1024
seg_len = seq_len // 2 - 1

num_hashes = 2
bucket_size = 64

manual_grad = False
is_full = False

if manual_grad and not log_dir_tmpl.endswith("_manual"):
    log_dir_tmpl += "_manual"
if is_full:
    log_dir = log_dir_full_attn_tmpl.format(seq_len)
else:
    log_dir = log_dir_tmpl.format(seq_len, num_hashes, bucket_size)    
log_dir

'./tf-reformer/dupltask/log_dir/lsh_seq1024_nr2_bs64'

# Train

In [7]:
tf.reset_default_graph()

xs = tf.placeholder(tf.int32, shape=[N, seq_len])
ys = tf.slice(xs, [0, tf.shape(xs)[1]//2 + 1], [-1, -1])
lr = tf.placeholder(tf.float32)

model = DuplTaskReformer(d_model, d_ff, num_heads, vocab_size, num_blocks, seq_len, dropout_rate=0.0, is_training=True,
                         num_hashes=num_hashes, bucket_size=bucket_size, is_full=is_full)
loss, train_op, _ = model.train(xs, ys, lr, manual_grad=manual_grad)
model.build_for_ar_gen(1)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.




In [8]:
for var in tf.trainable_variables():
  print(var)

<tf.Variable 'weight_mat:0' shape=(64, 128) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/qk_prj/dense/kernel:0' shape=(128, 128) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/qk_prj/dense/bias:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/dense/kernel:0' shape=(128, 128) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/dense/bias:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/ln/beta:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'num_blocks_0/multihead_lsh_attention/ln/gamma:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'num_blocks_0/positionwise_feedforward/W1:0' shape=(128, 256) dtype=float32_ref>
<tf.Variable 'num_blocks_0/positionwise_feedforward/B1:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'num_blocks_0/positionwise_feedforward/W2:0' shape=(256, 128) dtype=float32_ref>
<tf.Variable 'num_blocks_0/positionwise_feedforward/B2:0

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev_step = model.load(sess, log_dir) or 0
    
    max_iter = 150000
    losses = []
    start = time()
    
    print_every = 500
    eval_every = 1000
    save_every = 10000
    
    cur_loss = np.inf
    for it in range(prev_step+1, max_iter+1):
        _xs = np.stack([get_sample(vocab_size, seg_len) for _ in range(N)])
        _ys, _loss, _ = sess.run([ys, loss, train_op], feed_dict={xs:_xs, lr:1e-3})

        losses.append(_loss)
        
        if it % print_every == 0:        
            end = time()
            cur_loss = np.mean(losses)
            print("step:{0:} \telapsed: {1:.2f}s \t loss: {2:.3f}".format(it, end-start, cur_loss))
            losses = []
            start = end

        if cur_loss < 5 and it % eval_every == 0:
            start_infer = time()
            gen_data_samples = np.stack([get_sample(vocab_size, seg_len) for _ in range(1)])
            gen_sample = model.ar_gen(sess, seg_len, samples=gen_data_samples)
            print("left:", gen_sample[0][:seg_len+1])
            print("right:", gen_sample[0][seg_len+1:])
            
            left_seg = gen_sample[:, 1:seg_len+1]
            right_seg = gen_sample[:, seg_len+2:]
            acc = np.sum(left_seg == right_seg, -1)/seg_len
            avg_acc = np.mean(acc)
            print("accuracy: {0:.4f}, elapsed: {1:.2f}s".format(avg_acc, time()-start_infer))            
            
        if it % save_every == 0:
            model.save(sess, log_dir, it)

	Checkpoint found: ./tf-reformer/dupltask/log_dir/lsh_seq1024_nr2_bs64/model.ckpt-130000
	INFO:tensorflow:Restoring parameters from ./tf-reformer/dupltask/log_dir/lsh_seq1024_nr2_bs64/model.ckpt-130000


KeyboardInterrupt: ignored

Test

In [10]:
sample_batch_size = 1
num_batch = 10

In [11]:
tf.reset_default_graph()

train_nr = 2
eval_nr = 4
is_full = False

model = DuplTaskReformer(d_model, d_ff, num_heads, vocab_size, num_blocks, seq_len, is_training=False,
                         num_hashes=eval_nr, bucket_size=bucket_size, is_full=is_full)
model.build_for_ar_gen(sample_batch_size)

In [12]:
avg_acc_arr = []
start_infer = time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.load(sess, log_dir)
    for it in range(num_batch):
        tmp = time()
        
        eval_data = np.stack([get_sample(vocab_size, seg_len) for _ in range(sample_batch_size)])
        gen_sample = model.ar_gen(sess, seg_len, samples=eval_data)

        left_seg = gen_sample[:, 1:seg_len + 1]
        right_seg = gen_sample[:, seg_len + 2:]
        acc = np.sum(left_seg == right_seg, -1) / seg_len
        avg_acc = np.mean(acc)
        avg_acc_arr.append(avg_acc)
        if it == 0:
            elapsed = time() - tmp
            estimated = elapsed * num_batch
            print("elapsed per batch: {0:.2f}s, estimated time: {1:.2f}s".format(elapsed, estimated))

    print("left:", gen_sample[0][:seg_len + 1])
    print("right:", gen_sample[0][seg_len + 1:])
    print("accuracy: {0:.4f}, elapsed: {1:.2f}s".format(np.mean(avg_acc_arr), time() - start_infer))            

	Checkpoint found: ./tf-reformer/dupltask/log_dir/lsh_seq1024_nr2_bs64/model.ckpt-130000
	INFO:tensorflow:Restoring parameters from ./tf-reformer/dupltask/log_dir/lsh_seq1024_nr2_bs64/model.ckpt-130000
elapsed per batch: 17.29s, estimated time: 172.93s
left: [ 0  4 23 14 53 13  4 26 29  9 49 15 19  2 34  5 30 27 43 37 49 47 53 13
  4 59 48 48 48 46 14 46 40  6 40 32 22 28 16 20  7 62 57 26 29 52 48 15
 37 50 10 33 56 21 54  1 15 60 10 54 40 19 59  7 51  8 36 59 41 42 25 54
 20 10 31 54 56 33 16 40 48 38 54 16 39 47 39 14 62 61 28  2 36 50 20 57
 14 62  3 36 27 39  7 41 11 59 35 50 35 25 17 24 28 12  3 59 56 26 38 62
 13 57 51  2 56 38 22 30 31 40 34 55 54 39  3 45 59 20 55 38  8 49 62 44
 35 57 51  9 52 51 41 61 21 30  2  2 59 16 44 49 31 26 15 26  7 44 30 43
 58 54  3 38 33 37 23 24 15 34  4 49 56 54 45 33 43 48 45 36 46 42 29 18
  7  8  6 14 52 21 45 24 40 58  5 42 43 55 30 39 48 31 55  7 19  1 14 11
 56 22 59 60 23 31  4  5  5  1 48 61 35 10 57 18 37  8 27  5  1 24 51 19
 47 56 23 1